In [1]:
import pandas as pd
import numpy as np
#data = pd.read_csv('data/Compressed Mortality, 2016.txt', sep=" ", header=None)
#data.columns = ["a", "b", "c", "etc."]
#data = pd.read_fwf('data/Compressed Mortality, 2016.txt')


from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
% matplotlib inline

import glob
import os

In [2]:
# Compressed Mortality

# get a list of the data files the direction
onlyfiles = [f for f in listdir('data/Compressed Mortality/') if isfile(join('data/Compressed Mortality/', f))]
print(onlyfiles)

# for each txt file...
for i in np.arange(len(onlyfiles)):

    data_temp = pd.read_table('data/Compressed Mortality/' + onlyfiles[i], header='infer', index_col=None,  names=('Notes', 'County', 'County Code', 'Deaths', 'Population', 'Crude Rate'))
    #data_temp.drop(data_temp.index[0])
    data_dropped = data_temp.iloc[1:] #Remove the first row because it's the files's header
    data_dropped['County Code'] = pd.to_numeric(data_dropped['County Code']) # Make the zipcode a float instead of a string
    #print(data_dropped.head(5))

    data_dropped.insert(0, 'year', int(onlyfiles[i][-8:-4])) # Add the year as a column
#print(data.head)
    if i == 0:
        data = data_dropped
    else:
        data = pd.concat([data, data_dropped]) # add each year's text file to the overall dataframe


data_sorted = data.sort_values(by=['County Code', 'year']) # put in order

#print(data_sorted.head(5))

data_mortality = data_sorted


data_mortality.set_index(['County Code', 'year'], inplace=True, drop =False)


temp = np.zeros((data_mortality.shape[0],1))
unreliable = np.zeros((data_mortality.shape[0],1))
for i in range(data_mortality.shape[0]):
    
    
    #print(data['Crude Rate'].iloc[i])
    #s = re.findall('\d*\.?\d+', data['Crude Rate'].iloc[i])
    try:
        temp[i] = float(data_mortality['Crude Rate'].iloc[i].split()[0])
        unreliable[i] = 1
    except:
        temp[i] = float(data_mortality['Crude Rate'].iloc[i])
        unreliable[i] = 0
        
data_mortality['Crude Rate'] =temp
#data = data.assign(unreliable_deat_rate=unreliable)


#data.plot(kind='scatter', x='Opioid Perscription Rate', y='Crude Rate')

#print(data_mortality.head(20))

#print(data.iloc[2000])

['Compressed Mortality, 2010.txt', 'Compressed Mortality, 2004.txt', 'Compressed Mortality, 2005.txt', 'Compressed Mortality, 2011.txt', 'Compressed Mortality, 2007.txt', 'Compressed Mortality, 2013.txt', 'Compressed Mortality, 2012.txt', 'Compressed Mortality, 2006.txt', 'Compressed Mortality, 2002.txt', 'Compressed Mortality, 2016.txt', 'Compressed Mortality, 2003.txt', 'Compressed Mortality, 2015.txt', 'Compressed Mortality, 2001.txt', 'Compressed Mortality, 2000.txt', 'Compressed Mortality, 2014.txt', 'Compressed Mortality, 1999.txt', 'Compressed Mortality, 2008.txt', 'Compressed Mortality, 2009.txt']


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [3]:
# perscrption drugs

# Get all the relevant csv files
onlyfiles = [f for f in listdir('data/Prescriptions/') if isfile(join('data/Prescriptions/', f))]

print(onlyfiles)

# For all the files...
for i in np.arange(len(onlyfiles)):
    # load the CSV
    data_temp = pd.read_csv('data/Prescriptions/' + onlyfiles[i], index_col=None,  names=('State', 'County Code', 'Opioid Perscription Rate'))
    
    # Insert the year as a column in the dataframe
    data_temp.insert(0, 'year', int(onlyfiles[i][-8:-4]))
#print(data.head)
    if i == 0:
        data = data_temp
    else:
        data = pd.concat([data, data_temp]) # Add to the overall dataframe


data_sorted = data.sort_values(by=['County Code', 'year'])


data_perscriptions = data_sorted

data_perscriptions.set_index(['County Code', 'year'], inplace=True, drop =False)

['rxrate_2006.csv', 'rxrate_2012.csv', 'rxrate_2013.csv', 'rxrate_2007.csv', 'rxrate_2011.csv', 'rxrate_2010.csv', 'rxrate_2014.csv', 'rxrate_2015.csv', 'rxrate_2016.csv', 'rxrate_2009.csv', 'rxrate_2008.csv']


In [4]:
# county health statistics

def create_fips_code(df):
    fips_code = []
    for i in range(df['statecode'].shape[0]):
        # combine the statecode and countycode to create the fips_code
        string = '{:d}{:03d}'.format(df['statecode'].iloc[i], df['countycode'].iloc[i])
        fips_code.append(int(string)) 
    return fips_code

# load all data into a single dataframe
data_dir = 'data/county health rankings'
file_wildcard = os.path.join(data_dir, '*.csv')
for i, filepath in enumerate(reversed(glob.glob(file_wildcard))):
    year = filepath.split('/')[-1].split('\\')[-1].split(' ')[0]
    print(i, filepath, year)
    
    # load the data
    data = pd.read_csv(filepath)
    # convert all columns to lower case
    data.columns = map(str.lower, data.columns)
    # calculate the fips_code and insert as 1st column
    data.insert(loc=0, column='County Code', value=create_fips_code(data))
    # insert the year as the 2nd column
    data.insert(loc=1, column='year', value=int(year))
    #
    if i == 0:
        df = data
        orig_columns = data.columns
    else:
        df = pd.concat([df, data])
    print(data.shape)

    
#print(df.shape)

data_health_statistics = df
#data_health_statistics.rename(columns = {'5-digit fips code':'County Code'}, inplace = True)


data_health_statistics = data_health_statistics.sort_values(by=['County Code', 'year'])

data_health_statistics.set_index(['County Code', 'year'], inplace=True, drop =False)

0 data/county health rankings/2017 CHR analytic data.csv 2017
(3186, 358)
1 data/county health rankings/2015 CHR analytic data.csv 2015
(3191, 331)
2 data/county health rankings/2016 CHR analytic data.csv 2016
(3191, 356)
3 data/county health rankings/2014 CHR analytic data.csv 2014
(3191, 326)


In [5]:
#data_health_statistics.iloc[10]

In [13]:
# combine the three dataframes

data_temp = data_mortality.join(data_perscriptions, how ='outer', lsuffix = '_data_mortality', rsuffix = '_data_perscriptions')

data = data_temp.join(data_health_statistics, how = 'outer', lsuffix = '_data', rsuffix = '_data_health_statistics')

data.head(1005)


year_data_mortality Notes                       County  \
County Code year                                                           
1000        2014                  NaN   NaN                          NaN   
            2015                  NaN   NaN                          NaN   
            2016                  NaN   NaN                          NaN   
            2017                  NaN   NaN                          NaN   
1001        2006                  NaN   NaN                          NaN   
            2007                  NaN   NaN                          NaN   
            2008                  NaN   NaN                          NaN   
            2009                  NaN   NaN                          NaN   
            2010                  NaN   NaN                          NaN   
            2011                  NaN   NaN                          NaN   
            2012                  NaN   NaN                          NaN   
            2013                  NaN   NaN                          NaN   
            2014                  NaN   NaN                          NaN   
            2015                  NaN   NaN                          NaN   
            2016                  NaN   NaN                          NaN   
            2017                  NaN   NaN                          NaN   
1003        1999               1999.0   NaN           Baldwin County, AL   
            2000               2000.0   NaN           Baldwin County, AL   
            2001               2001.0   NaN           Baldwin County, AL   
            2002               2002.0   NaN           Baldwin County, AL   
            2003               2003.0   NaN           Baldwin County, AL   
            2004               2004.0   NaN           Baldwin County, AL   
            2005               2005.0   NaN           Baldwin County, AL   
            2006               2006.0   NaN           Baldwin County, AL   
            2007               2007.0   NaN           Baldwin County, AL   
            2008               2008.0   NaN           Baldwin County, AL   
            2009               2009.0   NaN           Baldwin County, AL   
            2010               2010.0   NaN           Baldwin County, AL   
            2011               2011.0   NaN           Baldwin County, AL   
            2012               2012.0   NaN           Baldwin County, AL   
...                               ...   ...                          ...   
2105        2016                  NaN   NaN                          NaN   
2110        2006                  NaN   NaN                          NaN   
            2007                  NaN   NaN                          NaN   
            2008                  NaN   NaN                          NaN   
            2009                  NaN   NaN                          NaN   
            2010                  NaN   NaN                          NaN   
            2011                  NaN   NaN                          NaN   
            2012                  NaN   NaN                          NaN   
            2013                  NaN   NaN                          NaN   
            2014                  NaN   NaN                          NaN   
            2015                  NaN   NaN                          NaN   
            2016                  NaN   NaN                          NaN   
            2017                  NaN   NaN                          NaN   
2122        2004               2004.0   NaN  Kenai Peninsula Borough, AK   
            2005               2005.0   NaN  Kenai Peninsula Borough, AK   
            2006                  NaN   NaN                          NaN   
            2007                  NaN   NaN                          NaN   
            2008               2008.0   NaN  Kenai Peninsula Borough, AK   
            2009               2009.0   NaN  Kenai Peninsula Borough, AK   
            2010                  NaN   NaN                       

In [7]:
## prescription data
rx14 = data[['statecode', 'Opioid Perscription Rate']][data['year']==2014]
rx15 = data[['statecode', 'Opioid Perscription Rate']][data['year']==2015]
rx16 = data[['statecode', 'Opioid Perscription Rate']][data['year']==2016]

rx14_statesum = rx14.groupby('statecode').sum()
rx15_statesum = rx15.groupby('statecode').sum()
rx16_statesum = rx16.groupby('statecode').sum()

#print(rx14)

In [45]:
#dCR = data[['Crude Rate']]

#print(dCR.index.get_level_values('year'))
#print(dCR)

#print(dCR[0])

state_codes = np.array(range(1, 58))
print(state_codes)
print(np.argwhere(state_codes >= 56.01)[0])

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57]
[56]


In [47]:
## all 50 states, death rate
## this code takes 10 seconds or so to run

dCR = data[['Crude Rate']]
years = range(1999, 2017)

total_death_yr_all = np.zeros((56, len(years)))

print(len(dCR))

state_codes = np.array(range(1, 58))

all_years = dCR.index.get_level_values('year')
all_county_codes = dCR.index.get_level_values('County Code')

for i in range(len(dCR)):
    
    this_county_code = all_county_codes[i]
    countycode2state = this_county_code / 1000
    idx_state = np.argwhere(state_codes >= countycode2state)[0]
    
    this_CR = dCR['Crude Rate'][i]

    if not isinstance(this_CR, np.float64):
        continue

    if np.isnan(this_CR):
        continue
    
    if not np.isnan(this_CR):

        this_year = all_years[i]
        if this_year in years:
            idx_yr = years.index(this_year)

            total_death_yr_all[idx_state-1, idx_yr] += this_CR

print(total_death_yr_all)

41002
[[  23.7   33.3   22.3 ...  369.   327.9  333.3]
 [   8.1    8.1   13.6 ...   53.8   71.2   31.3]
 [   0.     0.     0.  ...    0.     0.     0. ]
 ...
 [   0.    23.6  129.5 ... 1010.7 1097.7 1363.8]
 [  22.8   41.7   37.5 ...  275.8  334.6  404.5]
 [   0.     0.     0.  ...   98.4   70.5   60.6]]


In [49]:
## all 50 states, prescription rate
## this code takes 10 seconds or so to run

dCR = data[['Opioid Perscription Rate']]
years = range(2006, 2017)

total_rx_yr_all = np.zeros((56, len(years)))

print(len(dCR))

state_codes = np.array(range(1, 58))

all_years = dCR.index.get_level_values('year')
all_county_codes = dCR.index.get_level_values('County Code')

skipN = 0
for i in range(len(dCR)):
    
    this_county_code = all_county_codes[i]
    countycode2state = this_county_code / 1000
    idx_state = np.argwhere(state_codes >= countycode2state)[0]
    
    this_CR = dCR['Opioid Perscription Rate'][i]

    if not isinstance(this_CR, np.float64):
        skipN += 1
        continue

    if np.isnan(this_CR):
        continue
    
    if not np.isnan(this_CR):

        this_year = all_years[i]
        if this_year in years:
            idx_yr = years.index(this_year)

            total_rx_yr_all[idx_state-1, idx_yr] += this_CR

print(skipN)
print(total_rx_yr_all)


41002
2249
[[ 6649.6  6968.8  7303.2  7698.5  7824.8  8012.2  8595.6  8701.1  8422.4
   7731.1  7540. ]
 [  570.9   581.9   670.4   610.6   634.4   575.5   558.6   567.9   579.3
    502.7   431.1]
 [    0.      0.      0.      0.      0.      0.      0.      0.      0.
      0.      0. ]
 [  986.4  1059.7  1120.9  1154.8  1235.3  1277.1  1180.   1135.8  1126.1
   1064.9  1000.5]
 [ 5587.9  6482.5  6251.4  6907.5  6988.6  6871.2  7570.6  7556.6  8071.2
   7575.6  7852.7]
 [ 4310.9  4499.6  4681.5  4755.5  4886.   4860.8  4937.1  4783.6  4531.2
   4109.1  3838.1]
 [    0.      0.      0.      0.      0.      0.      0.      0.      0.
      0.      0. ]
 [ 3353.   3471.7  3529.6  3865.9  3805.9  4215.4  3990.7  4019.7  3771.4
   3644.8  3236.9]
 [  521.5   543.5   556.7   554.6   564.    569.7   571.8   560.3   553.7
    524.2   472.8]
 [  268.2   285.2   293.6   300.1   314.1   312.5   295.4   293.5   289.4
    262.8   243.7]
 [   31.7    41.2    34.5    34.4    37.1    39.8    40.3    

In [ ]:
plt.figure()
plt.scatter(data['Opioid Perscription Rate'], data['Crude Rate'])


In [ ]:
df14 = data[['statecode', 'Crude Rate']][data['year']==2014]
df15 = data[['statecode', 'Crude Rate']][data['year']==2015]
df16 = data[['statecode', 'Crude Rate']][data['year']==2016]

df14_statesum = df14.groupby('statecode').sum()
df15_statesum = df15.groupby('statecode').sum()
df16_statesum = df16.groupby('statecode').sum()

diff1514 = df15_statesum-df14_statesum
diff1615 = df16_statesum-df15_statesum

plt.figure()
plt.scatter(df14_statesum, df15_statesum)
plt.grid()
plt.plot([0, 2000], [0, 2000])
plt.xlabel('2014 deaths')
plt.ylabel('2015 deaths')

plt.figure()
plt.scatter(df15_statesum, df16_statesum)
plt.grid()
plt.plot([0, 2000], [0, 2000])
plt.xlabel('2015 deaths')
plt.ylabel('2016 deaths')

plt.figure()
plt.scatter(diff1514, diff1615)
plt.grid()
plt.plot([-100, 700], [-100, 700])
plt.xlabel('2014-2015 change in deaths')
plt.ylabel('2015-2016 change in deaths')

print(df14_statesum)
print(df15_statesum)
print(df16_statesum)



In [ ]:
## percent analysis
# but we are already normalizing by 100,000 people
# plus, 20 more people dying per 100,000 people is the same no matter what percent increase it is

diff1514_perc = np.divide(diff1514, df14_statesum)
diff1615_perc = np.divide(diff1615, df15_statesum)

#print(diff1514_perc)

plt.figure()
plt.scatter(diff1514_perc*100, diff1615_perc*100)
plt.grid()
plt.plot([-150, 600], [-150, 600])
plt.xlabel('2014-2015 perc change in deaths')
plt.ylabel('2015-2016 perc change in deaths')

state_num = 30
print(df14_statesum.loc[state_num])
print(df15_statesum.loc[state_num])
print(df16_statesum.loc[state_num])
print(diff1514_perc.loc[state_num])
